Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [transparency.ri.gov](http://www.transparency.ri.gov/payroll/).
<hr>

# Governor's Office Change

This Notebook contains code releated to the data cleaning for a breakdown of payroll changes in the 2 most recent months of the Governor's Office, during the transition from Gina Raimondo to Daniel McKee. The data is released quarterly, so this consists of data from January 2021, and April 2021. One issue with the data is inconsistant labeling of terminations, there was 3 employees who were terminated last quarter but the termination was removed, however the "total" pay change during the period was 0. This was handeled with the `fill_terminated` function which fills terminations with the prior quarter.

Another issue with the states payroll data is it lacks any sort of `employee_id` or a key to compare data over fiscal year, or quarter. To deal with this I created a `uid` columns which is `"first" + "m" + "last"`, while this works fine for a tiny < 100 persons governors office, i'm sure would cause duplicates in full payroll data of every department.
<hr>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# data cleaning
def start_pipeline(df):
    return df.copy()

def clean_columns(df):
    cols = [x.strip() for x in list(df)]
    df.columns = cols
    
    return df
    
def unique_name_key(df):
    """
    without any sort of "employee id" or key to lookup, the best we have is
    labeling people by first-m-last name. This works fine for just the GO, 
    however when used with all departments you run into some duplicates. 
    """
    df['uid'] = df['first'] + df['m'] + df['last']
    
    return df
    
def add_new_hires(df, clean_old=None):
    """
    last_quarter: all employees last quarter
    holdover:     a employee that started last quarter and is still on in recent quarter
    new:          a employee that started in the most recent quarter
    """
    if clean_old is None:
        df['period'] = 'last_quarter'
        return df
    else:
        holdover = [x for x in df['uid'].tolist() if x in clean_old['uid'].tolist()]
        new = [x for x in df['uid'].tolist() if x not in clean_old['uid'].tolist()]
        
        df.loc[df['uid'].isin(holdover), 'period'] = 'holdover'
        df.loc[df['uid'].isin(new), 'period'] = 'new'
        
        return df
    
def clean_column_names(df):
    cols = ['department', 'title', 'uid', '', 'termination']
    for col in cols:
        df[col] = df[col].str.strip()
        df[col] = df[col].replace(r'^\s+$', np.nan, regex=True)
        df[col] = df[col].replace('', np.nan)
        
    return df
    
    
def remove_columns(df):
    remove = ['first', 'm', 'last']
    df = df.drop(columns=remove)
    
    remove = ['regular', 'overtime', 'other']
    df = df.drop(columns=remove).dropna(how='all', axis=1)
    
    return df

def order_columns(df):
    cols = [
        'uid',
        'title',
        'department',
        'total',
        'annual',
        'period',
        'fiscal_year',
        'date_scraped'
    ]
    
    if 'termination' in list(df):
        cols.insert(3, 'termination')
    
    return df[cols].sort_values(by='annual', ascending=False)
    

def clean_raw_pipeline(df, old_ids=None):
    return (df
        .pipe(start_pipeline)
        .pipe(clean_columns)
        .pipe(unique_name_key)
        .pipe(clean_column_names)
        .pipe(add_new_hires, old_ids)
        .pipe(remove_columns)
        .pipe(order_columns)
    )

In [3]:
def start_pipeline(df):
    df = df.reset_index(drop=True)
    return df.copy()

def fill_terminated(df):
    """
    For some reason a few people that were terminated last quarter were no longer listed
    as terminated. However their "total" pay hasn't changed so we fill the termination date
    with last quarters date.
    """
    df['termination'] = df.groupby(['uid'])['termination'].fillna(method='ffill')
    
    return df

def clean_merged_pipeline(df):
    return (df
        .pipe(start_pipeline)
        .pipe(fill_terminated)
    )

In [4]:
gina_df = pd.read_csv('./data/raw/gina_exit.csv', parse_dates=['date_scraped'])
mckee_df = pd.read_csv('./data/raw/mckee.csv', parse_dates=['date_scraped'])

gina = clean_raw_pipeline(gina_df)
mckee = clean_raw_pipeline(mckee_df, gina)

combined = pd.concat([gina, mckee])
df = clean_merged_pipeline(combined)

In [5]:
# gina stats
gina_stats = df[(df['period']=='last_quarter') & (df['termination'].isnull())]

gina_stats = {
    "total_employees": gina_stats.shape[0],
    "highest_paid": gina_stats['annual'].max(),
    "total_annual": gina_stats['annual'].sum(),
    "median_annual": gina_stats['annual'].median(),
    "missing_title": gina_stats['title'].isnull().sum()
    
}

In [6]:
# mckee new employees
new_hire = df[(df['period']=='new')]
new_hire = new_hire[['uid', 'title', 'annual', 'period', 'date_scraped']]

# mckee terminated
termination = df[(df['period']!='last_quarter') & (df['termination'].notnull())].copy()
termination['termination'] = pd.to_datetime(termination['termination'])

termination.loc[termination['termination'] < '2021-03-02', 'termination_when'] = 'br'
termination.loc[termination['termination'] == '2021-03-02', 'termination_when'] = 'r'
termination.loc[termination['termination'] > '2021-03-02', 'termination_when'] = 'ar'

termination = termination.sort_values(by='termination')
termination = termination[['uid', 'title', 'annual', 'termination', 'termination_when', 'date_scraped']]

# mckee stats
mckee_stats = df[(df['period']!='last_quarter') & (df['termination'].isnull())]

mckee_stats = {
    "total_employees": mckee_stats.shape[0],
    "highest_paid": mckee_stats['annual'].max(),
    "total_annual": mckee_stats['annual'].sum(),
    "median_annual": mckee_stats['annual'].median(),
    "missing_title": mckee_stats['title'].isnull().sum()
}

stats_df = pd.DataFrame({"mckee": mckee_stats, "gina": gina_stats}).T

In [7]:
# raises
df_raises = df[df['termination'].isnull()]
df_raises = df_raises.groupby(['uid', 'date_scraped'])['annual'].sum()
diff = df_raises.groupby(['uid']).diff()
pct_diff = df_raises.groupby(['uid']).pct_change()

raises = pd.concat([df_raises, diff, pct_diff], axis=1)
raises.columns = ['current_annual', 'difference', '%_diff']

# get table of holdovers pay change
raises = (raises
    .groupby('uid')
    .filter(lambda x: len(x) == 2)
    .dropna(axis=0, how='any')
    .reset_index()
    .sort_values(by='difference', ascending=False)
)

# make pretty tables
raises_table = raises.drop(columns=['date_scraped'])
gina_merge = gina[['uid', 'annual']].rename(columns={"annual": "prior_quarter"})
raises_table = raises_table.merge(gina_merge, on='uid')

cols = list(raises_table)
cols.remove('prior_quarter')
cols.insert(1, 'prior_quarter')
raises_table = raises_table[cols]

no_raise = raises_table[raises_table['difference'] == 0]
increased = raises_table[raises_table['difference'] > 0]

In [8]:
df.to_csv('./data/clean/all_data_extract.csv', index=False)
stats_df.to_csv('./data/clean/summary_stats.csv')
increased.to_csv('./data/clean/pay_increased.csv', index=False)
no_raise.to_csv('./data/clean/no_raise.csv', index=False)
termination.to_csv('./data/clean/termination.csv', index=False)
new_hire.to_csv('./data/clean/new_hire.csv', index=False)